### Neccessary Imports

In [ ]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.metrics import confusion_matrix
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import ResNet50
from keras.layers import GlobalAveragePooling2D, Dense
from keras.models import Model, load_model

### Constants

In [ ]:
path = '../MLFinalProject/InsectDataset'
batch_size = 100

### Image Data Generation

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2,
                                   rotation_range=40, horizontal_flip=True,
                                   fill_mode='nearest')

train_gen = train_datagen.flow_from_directory(path, target_size=(224, 224),
                    class_mode='categorical', batch_size=batch_size, subset='training')

val_gen = train_datagen.flow_from_directory(path, target_size=(224, 224),
                class_mode='categorical', batch_size=batch_size, subset='validation')

print(train_gen.class_indices)
print(val_gen.class_indices)

### Model Creation

In [ ]:
def create_model(pretrained=True):
    if pretrained:
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    else:
        base_model = ResNet50(weights=None, include_top=False, input_shape=(224, 224, 3))

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(5, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model_pretrained = create_model(pretrained=True)
model_non_pretrained = create_model(pretrained=False)

### Model Summaries

In [ ]:
model_pretrained.summary()

In [ ]:
model_non_pretrained.summary()

### Pretrained Model Training

In [ ]:
steps, val_steps = train_gen.n//batch_size, val_gen.n//batch_size
num_epochs = 100

history_pretrained = model_pretrained.fit(train_gen, validation_data=val_gen,
                                          epochs=num_epochs, steps_per_epoch=steps,
                                          validation_steps=val_steps)

### Non-Pretrained Model Training

In [ ]:
history_non_pretrained = model_non_pretrained.fit(train_gen, validation_data=val_gen,
                                                  epochs=num_epochs, steps_per_epoch=steps,
                                                  validation_steps=val_steps)

### Saving Models

In [ ]:
model_pretrained.save('./model_pretrained.h5')
model_non_pretrained.save('./model_non_pretrained.h5')

### Saving Histories

In [ ]:
with open('history_pretrained.pkl', 'wb') as file:
    pickle.dump(history_pretrained.history, file)

with open('history_non_pretrained.pkl', 'wb') as file:
    pickle.dump(history_non_pretrained.history, file)

### Testing Saving

In [ ]:
# Load histories
with open('../MLFinalProject/history_pretrained.pkl', 'rb') as file:
    pretrained_history = pickle.load(file)

with open('../MLFinalProject/history_non_pretrained.pkl', 'rb') as file:
    non_pretrained_history = pickle.load(file)

# Load models
pretrained_model = load_model('../MLFinalProject/model_pretrained.h5')
non_pretrained_model = load_model('../MLFinalProject/model_non_pretrained.h5')

### Plotting Accuracy and Loss

In [ ]:
# Function to save loss plot
def save_loss_plot(history_dict, filename, is_pretrained):
    loss = history_dict['loss']
    val_loss = history_dict['val_loss']
    epochs = range(len(loss))

    plt.figure(figsize=(8, 6))  # Adjust the figure size as needed
    plt.plot(epochs, loss, 'r', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Test loss')
    model_type = 'Pretrained' if is_pretrained else 'Non-Pretrained'
    plt.title(f'{model_type} Model - Training and Test Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.tight_layout()  # Adjust the layout
    plt.savefig(filename, bbox_inches='tight')  # Save the figure with tight bounding box
    plt.show()

    # Function to save accuracy plot
def save_accuracy_plot(history_dict, filename, is_pretrained):
    acc = history_dict['accuracy']
    val_acc = history_dict['val_accuracy']
    epochs = range(len(acc))

    plt.figure(figsize=(8, 6))  # Adjust the figure size as needed
    plt.plot(epochs, acc, 'r', label='Training accuracy')
    plt.plot(epochs, val_acc, 'b', label='Test accuracy')
    model_type = 'Pretrained' if is_pretrained else 'Non-Pretrained'
    plt.title(f'{model_type} Model - Training and Test Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.tight_layout()  # Adjust the layout
    plt.savefig(filename, bbox_inches='tight')  # Save the figure with tight bounding box
    plt.show()


save_accuracy_plot(pretrained_history, '../MLFinalProject/pretrained_accuracy.png', True)
save_loss_plot(pretrained_history, '../MLFinalProject/pretrained_loss.png', True)

save_accuracy_plot(non_pretrained_history, '../MLFinalProject/non_pretrained_accuracy.png', False)
save_loss_plot(non_pretrained_history, '../MLFinalProject/non_pretrained_loss.png', False)

### Confusion matrices

In [ ]:
def plot_confusion_matrix(model, generator):
    # Get predictions and true labels
    predictions = model.predict(generator)
    y_pred = np.argmax(predictions, axis=1)
    y_true = generator.classes

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred, labels=range(5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')

    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()

print("Confusion Matrix for Pretrained Model")
plot_confusion_matrix(model_pretrained, val_gen)

print("\nConfusion Matrix for Non-Pretrained Model")
plot_confusion_matrix(model_non_pretrained, val_gen)